In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json, os

In [ ]:
models = [
    ("<path>/Experiments/cholecfFixAbl_none_10_1.0_16_1_1.0_0.99_OctreeNCASegmentation", "OctreeNCA"),
    ("<path>/Experiments/cholec_M3dSegmentation/", "M3D-NCA"),
    ("<path>/Experiments/cholec_unet_UNetSegmentation/", "UNet"),
    ("<path>/Experiments/cholec_munet3d_UNet_resnet18_0_minUNet3DSegmentation/", "Resnet18"),
    ("<path>/Experiments/cholec_munet3d_UNet_densenet121_0_minUNet3DSegmentation/", "Densenet"),
    ("<path>/Experiments/cholec_sam2_base_plus_Sam3DSegmentation", "SAM2"),
]

CLASS_NAMES = ["Abdominal wall", "Liver", "Fat", "Grasper", "Gallbladder"]

In [ ]:
nan_template = pd.read_csv(f"<path>/Experiments/cholec_sam_Sam3DSegmentation/eval/standard.csv", sep='\t')
nan_template.set_index('Unnamed: 0', inplace=True)
annnotation_list = []
annotation_dict = {}
all_files = []
for i, (model, model_name) in enumerate(models):
    eval_file = {}

    score = pd.read_csv(f"{model}/eval/standard.csv", sep='\t')
    score.set_index('Unnamed: 0', inplace=True)
    score[nan_template.isna()] = np.nan
    

    for c in range(5):
        eval_file[CLASS_NAMES[c]] = round(score.loc[:, f"DiceScore/{c}"].mean() * 100, 2)
        
    score = score.loc[:, ["DiceScore/0", "DiceScore/1", "DiceScore/2", "DiceScore/3", "DiceScore/4"]]
    score = score.mean(axis=1)
    eval_file["mDice"] = round(score.mean() * 100, 2)

    mem_allocation = None
    if os.path.exists(f"{model}/mem_allocation.json"):
        mem_allocation = round(json.load(open(f"{model}/mem_allocation.json"))["byte"] / 1024**3, 2)
    num_params = json.load(open(f"{model}/num_params.json"))["num_params"]


    eval_file["model"] = model_name
    eval_file["mem_allocation"] = mem_allocation
    eval_file["num_params"] = num_params
    df = pd.DataFrame(index=[i], data=eval_file, columns=["model", "mem_allocation", "num_params", *CLASS_NAMES, "mDice"])
    all_files.append(df)

all_files = pd.concat(all_files)

/tmp/ipykernel_330848/1271043279.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_files = pd.concat(all_files)


In [4]:
from IPython.display import display, HTML
display(HTML(all_files.to_html()))

,model,mem_allocation,num_params,Abdominal wall,Liver,Fat,Grasper,Gallbladder,mDice
0,OctreeNCA,2.41,16960,70.62,76.92,74.64,46.23,56.68,64.01
1,M3D-NCA,13.33,12480,71.85,74.12,77.78,45.64,54.41,64.03
2,UNet,13.96,19073285,59.22,58.97,66.47,45.07,41.82,53.13
3,Resnet18,16.08,42656757,74.89,72.82,70.46,40.93,54.51,61.90
4,Densenet,22.44,31245621,70.51,74.25,76.82,42.62,55.42,63.33
5,SAM2,NaN,80833666,57.04,53.44,49.49,82.29,66.20,60.07


In [6]:
all_files_styled = all_files.style.format(thousands=',', na_rep="-", precision=2)
all_files_styled.hide(axis="index")
all_files_styled.to_latex("tables/cholec_split.tex")